Feriados/aniverssarios

Margin para a espiral

In [15]:


from unidecode import unidecode
import reportlab
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch, mm
from reportlab.pdfgen import canvas
from datetime import datetime, timedelta
from reportlab.lib.utils import ImageReader
from reportlab.lib.colors import HexColor
from reportlab.lib.colors import Color
import calendar
import math
import pandas as pd
import requests
from bs4 import BeautifulSoup
from ics import Calendar


In [16]:

# Processes city name for use in the holiday function
def process_city_name(city_name):
    """
    Processes a city name to prepare it for use in holiday lookup functions.

    Args:
    - city_name: String in the format 'City - State'

    Returns:
    - List containing processed city and state names.
    """
    city, state = city_name.split("-")
    city = city.strip().lower().replace(" ", "_")
    city = unidecode(city)
    state = state.strip().lower()
    return [city, state]

# Fetches the holidays of a specific year for a given city
def get_holidays(year, city_name):
    """
    Retrieves holidays for a given city and year from an external website.

    Args:
    - year: Year for which holidays are needed.
    - city_name: City name in the format 'City - State'.

    Returns:
    - List of holidays with date and description.
    """
    holidays = []
    city, state = process_city_name(city_name)
    url = f'https://www.feriados.com.br/feriados-{city}-{state}.php?ano={year}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    holiday_list = soup.find('ul', {'class': 'multi-column'})

    for holiday in holiday_list.find_all('div'):
        holiday_date, holiday_name = holiday.get_text().split(' - ')
        holiday_date = datetime.strptime(holiday_date, '%d/%m/%Y')

        if [holiday_date, holiday_name.lower()] not in [[date, name.lower()] for date, name in holidays]:
            holidays.append([holiday_date, holiday_name])
    return holidays

# Returns holidays for a specific day
def get_holidays_for_day(date, holidays):
    """
    Checks if a given date has holidays and returns their descriptions.

    Args:
    - date: Date object for the specific day.
    - holidays: List of holidays with dates and descriptions.

    Returns:
    - List of strings describing holidays for the specific day.
    """
    events = []
    for holiday in holidays:
        if (holiday[0].month, holiday[0].day) == (date.month, date.day):
            events.append("   - " + holiday[1])
    return events

# Writes the holidays for the day, breaking lines if needed
def write_holidays(canvas, x, y, holiday_str):
    """
    Draws holiday descriptions on the PDF, handling line breaks.

    Args:
    - canvas: ReportLab canvas object for drawing on the PDF.
    - x, y: Coordinates for the starting point of the text.
    - holiday_str: List of holiday descriptions to be written.

    Returns:
    - None
    """
    canvas.setFont("Helvetica", 12)
    num_holidays = len(holiday_str)
    for i in range(num_holidays):
        canvas.drawString(x, y + (1 - num_holidays + 1.8 * i) * 0.1 * inch, holiday_str[i])


In [17]:
def extract_birthdays_from_ics(file_path):
    """
    Extracts birthdays from an .ics file and returns them as a list of [datetime, name].

    Args:
        file_path (str): Path to the .ics file. If None, returns an empty list.

    Returns:
        list: List of [datetime_of_birthday, name_of_birthday].
    """
    if file_path is None:
        return []  # Return an empty list if the input is None
    
    birthdays = []
    
    # Read and parse the .ics file
    with open(file_path, 'r', encoding='utf-8') as file:
        calendar = Calendar(file.read())
    
    # Iterate through events in the calendar
    for event in calendar.events:
        # Extract the event name and date
        name = event.name
        date = event.begin.datetime  # Get the start date and time
        
        # Append to the list
        birthdays.append([date, name])
    
    return birthdays


In [18]:
#Essa função desenha uma linhha pontilhada de bolinahs
def draw_line_of_circles(pdf_canvas, x1, x2, y,radius, circle_spacing):
    # Set the color of the circles
    circle_color = Color(0.66, 0.65, 0.65)
    pdf_canvas.setFillColor(circle_color)


    # Calculate the number of circles to draw
    num_circles = int((x2 - x1) / circle_spacing)+1

    # Draw each circle
    for i in range(num_circles):
        x = x1+ circle_spacing/2 + (i * circle_spacing)

        pdf_canvas.circle(x, y, radius, stroke=0, fill=1)


In [19]:
#Essa função desenha os retângulos da págin de organização
def draw_rectangles(C, N,spiral_spacing):
    """
    Draws 2*N evenly spaced rectangles in two columns of the page.

    Args:
    - C: Canvas object
    - N: number of rectangles (will be doubled to draw 2*N rectangles)
    """
    # set canvas dimensions and margins
    margin = 0.7*inch

    width, height = C._pagesize
    center_width= 10
    v_spacing= 10
    rect_width = (width -center_width- 2*margin) / 2
    rect_height = (height - 4*margin) / N


    # set rectangle dimensions and style
    C.setFont("Helvetica-Bold", 24)
    C.drawString(margin, height - inch , "Organização")


    # draw rectangles
    for j in range(2):
        C.setFillColorRGB(0.8, 0.8, 0.8)
        C.setStrokeColorRGB(0.8, 0.8, 0.8)
        for i in range(N):
              # draw rectangle in left column
            x1 = margin+((-1)**(j+1))*spiral_spacing
            y1 = height- 1.5*inch - i*(rect_height+v_spacing)
            x2 = x1 + rect_width
            y2 = y1 - rect_height
            C.rect(x1, y1, x2-x1, y2-y1, stroke=i%2, fill=1-i%2)

            # draw rectangle in right column

            x1 = margin + center_width+ rect_width+((-1)**(j+1))*spiral_spacing
            x2 = x1 + rect_width
            C.rect(x1, y1, x2-x1, y2-y1, fill=i%2, stroke=1-i%2)
        if not j:
          C.showPage()



In [20]:
#essa função escreve a frase do dia, quebrando a linha no luhar adequado se o texto for muito longo

def write_phrase(c, x1, x2, y, text, fonte):
    # Define the font and size
    c.setFillColorRGB(0.2,0.2,0.2)
    c.setFont("Helvetica", 13)
    text_width = c.stringWidth(text)
    fonte_width = c.stringWidth(fonte)

    # Check if the text fits within the given x1 and x2 positions
    if text_width <= x2 - x1:
        c.drawString(x1, y, text)
        y -= 20
        c.drawString(x2-fonte_width, y, fonte)

    else:
        # Split the text into multiple lines
        lines = []
        line = ""
        for word in text.split():
            if c.stringWidth(line + word) <= x2 - x1:
                line += " " + word if line else word
            else:
                lines.append(line)
                line = word
        lines.append(line)

        # Write each line to the canvas
        for line in lines:
            c.drawString(x1, y, line)
            y -= 15  # move the current position down by 15 units (assuming 12pt font size)

        y -= 5
        c.drawString(x2-fonte_width, y, fonte)

#escreve uma string verticalmente
def write_vertical_text(canvas, text, x, y):
    canvas.saveState()
    canvas.translate(x, y)
    canvas.rotate(90)
    canvas.drawString(0, 0, text)
    canvas.restoreState()

In [21]:

#Desenha o mini calendário do mês
def draw_calendar(dt, c, x, y, s=1):
    # Set up month and year
    month = dt.month
    year = dt.year

    # Define abbreviated weekday names
    week_day_names = ['S', 'T', 'Q', 'Q', 'S', 'S', 'D']

    # Define cell dimensions and padding with scaling factor
    cell_width = 0.175 * inch * s
    cell_height = 0.175 * inch * s
    padding = 0.02 * inch * s
    header_height = 0.3 * inch * s

    # Set up font size with scaling factor
    font_size = 8 * s
    bold_font_size = 9 * s

    # Draw horizontal line below weekday names
    c.setStrokeColorRGB(0.7, 0.7, 0.7)

    line_y = y + header_height - 3 * padding *s
    c.line(x, line_y, x + 6.8 * cell_width, line_y)

    # Draw month and year header
    c.setStrokeColorRGB(0.2,0.2,0.2)
    c.setFont("Helvetica-Bold", bold_font_size)
    for i, day in enumerate(week_day_names):
        cell_x = x + i * cell_width + padding
        cell_y = y + header_height - 2 * padding
        c.drawString(cell_x, cell_y, day)

    # Get calendar data for the month
    cal = calendar.monthcalendar(year, month)

    # Draw calendar cells
    c.setFont("Helvetica", font_size)
    for i, week in enumerate(cal):
        for j, day in enumerate(week):
            if day == 0:
                continue
            cell_x = x + j * cell_width + padding
            cell_y = y + header_height - (i + 1) * cell_height - padding
            c.drawString(cell_x, cell_y, str(day))
            

#Desenha a página de mini calendários do ano
def draw_calendar_page(c, x, y, year, hollydays):
  width, height = letter
  m=1
  c.setFillColorRGB(0.2,0.2,0.2)
  c.setFont("Helvetica-Bold", 23)
  c.drawCentredString(x+3*inch,height-0.5*inch,str(year))
  for i in range(4):
    for j in range(3):
      dt = datetime(year=year, month=m, day=1)
      c.setFont("Helvetica-Bold", 12)
      c.drawString(x+0.70*inch-  c.stringWidth(meses[dt.month-1], "Helvetica-Bold", 12)/2  +2.3*j*inch, y-2.5*i*inch+0.5*inch, meses[dt.month-1])
      draw_calendar(dt, c, x+2.3*j*inch, y-2.5*i*inch, s=1.2)
      h_num=0
      for h in hollydays:
        if h[0].month==m:
         c.setFont("Helvetica", 9)
         c.drawString(x+2.3*j*inch,y-2.5*i*inch-1.1*inch-0.15*inch*h_num,str(h[0].day)+"  ")
         c.setFont("Helvetica", 8)
         c.drawString(x+2.3*j*inch+c.stringWidth(str(h[0].day)+"  ", "Helvetica", 9),y-2.5*i*inch-1.1*inch-0.15*inch*h_num,h[1])
         h_num+=1
      m+=1



In [31]:
#Desenha a página de projetos do planner
def projects_page(c,x,y):
  width, height = letter
  c.setFont("Helvetica-Bold", 20)
  c.drawString(x,height-inch,"PROJETOS")


  for i in range(4):

    # Draw a filled rectangle
    c.setFillColorRGB(0.85,0.85,0.85)

    c.rect(x, y, 5 * inch, -0.5 * inch, stroke=0, fill=1)
    c.setFillColorRGB(0.2,0.2,0.2)

    c.setFont("Helvetica", 14)
    c.drawString(x + 6.6 * inch-c.stringWidth("   /     /     ", "Helvetica", 17),y -0.20 * inch,"Prazo final")
    c.setFillColorRGB(0.7, 0.7, 0.7)
    c.setFont("Helvetica", 17)
    c.drawString(x + 6.6 * inch-c.stringWidth("     /     /     ", "Helvetica", 17),y -0.46 * inch,"     /     /    ")

    c.setFillColorRGB(0.6,0.6,0.6)
    write_vertical_text(c,  "Etapas",x-8, y  -1.6 * inch)

    # Draw lines with constant y coordinate
    for l in range(5):
        # Set stroke color for lines
        c.setStrokeColorRGB(0.7, 0.7, 0.7)
        c.setFillColorRGB(0.7, 0.7, 0.7)
        line_length =  5 * inch
        c.line(x, y  -0.5 * inch- 0.3 * (l+1) * inch, x + line_length, y  -0.5 * inch- 0.3 * (l+1) * inch)
        r=0.085*inch
        c.circle(x+r,y -0.5 * inch+1.4*r- 0.3 * (l+1) * inch,r)

        c.drawString(x + line_length-c.stringWidth("     /    /    ", "Helvetica", 15),y -0.45 * inch - 0.3 * (l+1) * inch,"    /    /   ")
        draw_line_of_circles(c, x + line_length, x + line_length+2.1* inch, y  -0.5 * inch- 0.3 * (l+1) * inch, 0.01*inch, 0.2*inch)
    y=y-2.3*inch



def horarios_page(c, x, y, start_time="08:00", end_time="18:00", interval=30):
    """
    Draws a weekly schedule page with 5 columns (Monday to Friday) and rows for time slots.

    Args:
        c: ReportLab canvas object.
        x (float): Starting x-coordinate for the table.
        y (float): Starting y-coordinate for the table.
        start_time (str): Start of the schedule in HH:MM format (e.g., "08:00").
        end_time (str): End of the schedule in HH:MM format (e.g., "18:00").
        interval (int): Interval in minutes for each time slot row.
    """
    width, height = letter

    # Header: "Horários"
    c.setFont("Helvetica-Bold", 20)
    c.drawString(x/2.15, height - inch, "HORÁRIOS")
    
    # Days of the week (Portuguese)
    days_of_week = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]

    # Time slots
    from datetime import datetime, timedelta
    start = datetime.strptime(start_time, "%H:%M")
    end = datetime.strptime(end_time, "%H:%M")
    time_slots = []
    current = start
    while current <= end:
        time_slots.append(current.strftime("%H:%M"))
        current += timedelta(minutes=interval)

    # Column width and row height
    col_width = 1.3 * inch  # Smaller columns to leave margins
    row_height = 0.5 * inch

    # Draw column headers (days of the week) with alternate shading
    c.setFont("Helvetica", 12)  # Smaller font for week names
    for i, day in enumerate(days_of_week):
        # Shade every odd column (0-based index means 0, 2, 4 are odd columns)
        if i % 2 == 0:
            c.setFillColorRGB(0.9, 0.9, 0.9)  # Set gray shade
            c.rect(x + i * col_width, y - row_height, col_width, -len(time_slots) * row_height, stroke=0, fill=1)
        c.setFillColorRGB(0, 0, 0)  # Reset color for text
        # Center the text
        text_x = x + i * col_width + col_width / 2 - c.stringWidth(day, "Helvetica-Bold", 12) / 2
        c.drawString(text_x, y - 0.15 * inch, day)  # Adjusted y position

    y -= row_height  # Move down for the first row

    # Draw first row (solid line)
    c.setStrokeColorRGB(0.6, 0.6, 0.6)
    c.setDash()  # Solid line
    c.line(x, y, x + 5 * col_width, y)

    # Draw rows (time slots) with dotted lines
    c.setFont("Helvetica", 12)
    y -= row_height  # Move down for the first row
    for time in time_slots[:-1]:  # Exclude the last time slot
        # Vertically center the time label between two rows
        time_y = y + row_height / 2
        c.drawString(x - 0.8 * inch, time_y, time)

        # Draw dotted lines for the row
        c.setDash(3, 2)  # Dotted line: 3 points on, 2 points off
        c.line(x, y, x + 5 * col_width, y)

        # Move down for the next row
        y -= row_height

    # Draw last row (solid line)
    c.setDash()  # Solid line
    c.line(x, y, x + 5 * col_width, y)

    # Draw the final time label
    c.drawString(x - 0.8 * inch, y + row_height / 2, time_slots[-1])

In [23]:
#Desenha as página de rascunhos do planner
def sketch_pages(c,N,spiral_spacing):

    for j in range(N):
      c.setFont("Helvetica-Bold", 20)
      c.setFillColorRGB(0.2, 0.2, 0.2)
      # Calculate the vertical position for the annotations
      page_width, page_height = letter
      margin = 0.7*inch
      annotation_text = "Anotações"
      annotation_x = margin-spiral_spacing
      annotation_y = page_height - 0.75 * inch  # You can adjust this value for the blank space

      # Write the annotations centered at the top of the page
      c.drawString(annotation_x, annotation_y, annotation_text)

      # Set the font for the horizontal lines
      c.setFont("Helvetica", 12)

      # Calculate the spacing between lines and draw 15 lines
      line_spacing = 0.35 * inch
      line_start_y = annotation_y - 0.70 * inch  # Adjust this value based on your preference

      c.setStrokeColorRGB(0.7, 0.7, 0.7)
      for i in range(14):
          line_y = line_start_y - i * line_spacing
          c.line(margin-spiral_spacing, line_y, page_width-margin-spiral_spacing, line_y)


      for i in range(13):
          line_y = line_start_y - (i+14) * line_spacing
          draw_line_of_circles(c, margin-spiral_spacing, page_width-margin-spiral_spacing, line_y, 0.01*inch, 0.2*inch)

      c.showPage()

      line_start_y = annotation_y  # Adjust this value based on your preference

      c.setStrokeColorRGB(0.7, 0.7, 0.7)
      for i in range(15):
          line_y = line_start_y - i * line_spacing
          c.line(margin+spiral_spacing, line_y, page_width-margin+spiral_spacing, line_y)
      for i in range(15):
          line_y = line_start_y - (i+14) * line_spacing
          draw_line_of_circles(c, margin+spiral_spacing, page_width-margin+spiral_spacing, line_y, 0.01*inch, 0.2*inch)

      c.showPage()

    c.setFont("Helvetica-Bold", 20)

    c.setFillColorRGB(0.2, 0.2, 0.2)
    # Calculate the vertical position for the annotations
    page_width, page_height = letter
    margin = 0.7*inch
    annotation_text = "Anotações"
    annotation_x = margin-spiral_spacing
    annotation_y = page_height - 0.75 * inch  # You can adjust this value for the blank space

    # Write the annotations centered at the top of the page
    c.drawString(annotation_x, annotation_y, annotation_text)

    # Set the font for the horizontal lines
    c.setFont("Helvetica", 12)

    # Calculate the spacing between lines and draw 15 lines
    line_spacing = 0.35 * inch
    line_start_y = annotation_y - 0.70 * inch  # Adjust this value based on your preference

    c.setStrokeColorRGB(0.7, 0.7, 0.7)
    for i in range(14):
        line_y = line_start_y - i * line_spacing
        c.line(margin-spiral_spacing, line_y, page_width-margin-spiral_spacing, line_y)

    for i in range(13):
        line_y = line_start_y - (i+14) * line_spacing
        draw_line_of_circles(c, margin-spiral_spacing, page_width-margin-spiral_spacing, line_y, 0.01*inch, 0.2*inch)


In [24]:
#Desenha a página de rosto do planner
def cover(c,year,spiral_spacing, image_path):

    width, height = letter
    c.setFont("Helvetica-Bold", 40)
    c.setFillColorRGB(0.2,0.2,0.2)
    c.drawCentredString((width-spiral_spacing)/2+spiral_spacing,height-2*0.7*inch, "PLANNER "+ str(year))

    margin=0.7*inch
    box_width=5.3* inch

    #set left margin
    x=(width-box_width)/2+spiral_spacing
    y=height-2*margin
    c.setFillColorRGB(0.85,0.85,0.85)

    #draw text rectangles
    c.rect(x, y-1.7*inch, box_width, -0.45 * inch, stroke=0, fill=1)
    c.rect(x, y-2.5*inch, box_width, -0.45 * inch, stroke=0, fill=1)
    c.rect(x, y-3.3*inch, 1.5 * inch, -0.45 * inch, stroke=0, fill=1)
    c.rect(x+1.9*inch, y-3.3*inch, 1.5 * inch, -0.45 * inch, stroke=0, fill=1)
    c.rect(x+3.8*inch, y-3.3*inch, 1.5 * inch, -0.45 * inch, stroke=0, fill=1)

    #write rectangle headers
    c.setFillColorRGB(0.2,0.2,0.2)
    c.setFont("Helvetica-Bold", 15)
    c.drawString(x, y-1.65*inch,"Nome")
    c.drawString(x, y-2.45*inch,"Contato")
    c.drawString(x, y-3.25*inch,"Signo")
    c.drawString(x+1.9*inch, y-3.25*inch,"Ascendente")
    c.drawString(x+3.8*inch, y-3.25*inch,"Lua")


    #github link
    c.setFont("Helvetica", 12)
    c.drawCentredString((width-spiral_spacing)/2+spiral_spacing,43, "Planner open-source disponível em:")
    c.drawCentredString((width-spiral_spacing)/2+spiral_spacing, 30, "github.com/turdutra/pdfgen_planner")

    #logo
    im_width=height*150/(205*4)
    c.drawInlineImage(image_path, x=(width-spiral_spacing)/2+spiral_spacing-im_width/2, y=76, width=im_width, height=height/4)




In [32]:
#Define os nomes em portugês dos dias da semana e do mês
dias_sem=["","Seg","Ter","Qua","Qui","Sex","Sáb","Dom"]
meses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']


#Função que cria o planner
def generate_weekly_planner(year,lista_cidades, aniversarios_ics_path=None):

    #importar as aspas da planilha
    aspas = pd.read_excel("aspas_"+str(year)+".xlsx", header=None, names=['Quote', 'Name'])

    #importar a imagem do logo
    image_path = 'urso.jpg'

    # Define the PDF canvas
    c = canvas.Canvas(str(year) + " Weekly Planner.pdf", pagesize=letter)
    width, height = letter
    margin = 0.7*inch
    center_width = margin/2
    column_width=(width-2*margin-center_width)/2
    header_height= 2.6 * margin
    line_height= margin*0.48
    spacing=margin/3
    spiral_spacing=0.17*inch

    # Define the font styles
    c.setFont("Helvetica", 12)
    c.setStrokeColorRGB(0.2,0.2,0.2)
    c.setFillColorRGB(0.2,0.2,0.2)

    # Define the starting date
    date = datetime(year, 1, 1)
    while date.weekday() != 0:
        date -= timedelta(days=1)
    # Loop through each week of the year

    #get the list of holidays

    holidays_list=get_holidays(year, lista_cidades)
    holidays_list_next=get_holidays(year+1, lista_cidades)

    birthday_list=extract_birthdays_from_ics(aniversarios_ics_path)

    cover(c,year,spiral_spacing,image_path)
    c.showPage()

    draw_calendar_page(c,1.6*margin,height - 0.8*header_height,year,holidays_list)
    c.showPage()
    
    draw_calendar_page(c,2.1*margin,height - 0.8*header_height,year+1,holidays_list_next)
    c.showPage()
    for _ in range(2):
      projects_page(c,margin,height - 0.8*header_height)
      c.showPage()
      horarios_page(c,2.15*margin,height - 0.8*header_height, start_time="08:00", end_time="20:00", interval=60)
      c.showPage()
    
    draw_rectangles(c,5,spiral_spacing)
    c.setFillColorRGB(0.2,0.2,0.2)
    c.setStrokeColorRGB(0.2,0.2,0.2)

    c.showPage()
    semana_num=0
    while date.year < year+1:

        # Draw the week heading
        c.setFont("Helvetica-Bold", 24)
        c.drawString(margin-spiral_spacing, height - 1.1*margin , meses[date.month-1])




        #draw 2 pages for the week
        for p in range(2):
          c.setStrokeColorRGB(0.7, 0.7, 0.7)
          # Draw the lines for each day
          for i in range(2):
            for j in range(2):
              num_lines= 12
              x1 = margin + j * (column_width+center_width)+((-1)**(p+1))*spiral_spacing
              x2 = margin + column_width + j * (column_width+center_width)+((-1)**(p+1))*spiral_spacing
              y = height - header_height - i* (num_lines* line_height + 3*spacing)
              c.setFillColorRGB(0.3,0.3,0.3)
              if i*2+j*1+4*p:
                c.setFont("Helvetica-Bold", 24)
                c.drawString(x1, y , str(date.day))
                num_lenght=c.stringWidth(str(date.day))
                c.setFont("Helvetica-Bold", 14)
                c.drawString(x1+num_lenght+margin/2, y , dias_sem[i*2+j*1+4*p])
                write_holidays(c, x1+num_lenght+margin, y , get_holidays_for_day(date,holidays_list+birthday_list))
                for k in range(num_lines):
                    c.setStrokeColorRGB(0.7, 0.7, 0.7)
                    c.line(x1, y-(k+1.3)*line_height , x2, y-(k+1.3)*line_height)

                date=date+timedelta(days=1)
              #draw the week square
              else:
                c.setFont("Helvetica-Bold", 18)
                week_str = "Semana " + str(semana_num)
                c.drawString(x1, y , week_str)
                #draw_smiley_face(c, x1, y-1.3*line_height, inch, semana_num)
                for k in range(num_lines):
                  if k<5:
                    c.setStrokeColorRGB(0.7, 0.7, 0.7)
                    c.line(x1, y-(k+1.3)*line_height , x2, y-(k+1.3)*line_height)
                    r=0.085*inch
                    c.circle(x1+r,y-(k+1.3)*line_height+1.3*r,r)
                  else:
                    draw_line_of_circles(c, x1, x2, y-(k+1.3)*line_height, 0.01*inch, 0.2*inch)



          #habit tracker
          #maybe put tiny calendar next to it
          #page 2
          if p:
            draw_calendar(date,c,width-2.8*margin+spiral_spacing,height-3*spacing)
            c.setFont("Helvetica-Bold", 14)
            r=0.085*inch
            c.setFillColorRGB(0.6,0.6,0.6)
            write_vertical_text(c,"Hábitos", 1.3*margin+spiral_spacing-8,height-2*spacing-2*line_height-2*r)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            for k in range(3):
              x1=1.3*margin+spiral_spacing
              x2= width-3.2*margin+spiral_spacing
              y=height-2*spacing-k*line_height-2*r
              c.setStrokeColorRGB(0.7, 0.7, 0.7)
              c.line(x1,y,x2,y)
              for l in range(7):
                c.circle(x2-l*2.3*r-r, y+1.3*r, r)

          #page 1
          else:
            write_phrase(c,3.7*margin-spiral_spacing,width-margin-spiral_spacing,height-margin,aspas['Quote'][semana_num],aspas['Name'][semana_num])


          # Add a new page if necessary

          c.showPage()

        semana_num+=1



    sketch_pages(c,5,spiral_spacing)

    # Save the PDF
    c.save()


In [33]:
Cidade_feriado="Campinas-SP"
Ano=2025
aniversario_ics="D:/Downloads/Birthdays_turdutra@gmail.com.ics"
generate_weekly_planner(Ano,Cidade_feriado,aniversario_ics)